![alttext](logo.png)

### Nombre Alumno: Eduardo López
### Generación :  G10
### Profesores :  Robert Escalante - Gianina Salomó 
### Fecha : Lunes 20 de abril de 2020

# Ejercicio 1: Ingesta de datos semiestructurados
* Utilizando las librerías requests y json, realize una consulta a la API BallDontLie https://www.balldontlie.io/.
* Mediante esta API, genere una consulta de 100 juegos (busquen el endpoint Get All Games en la página, dentro de Games). Puede usar la opción ?per_page=100 en la dirección de la API para lograr esto.
* Guarde la consulta en una variable, y asegúrese que ésta sea semiestructurada. Consulte los datos y metadatos asociados.
* Dentro de la lista de datos, comente cuáles son las llaves de cada registro.

In [1]:
import requests
import pandas as pd 
import numpy as np

In [2]:
response = requests.get('https://www.balldontlie.io/api/v1/games/?per_page=100')

In [3]:
response

<Response [200]>

In [4]:
to_df = response.json()

In [5]:
print("Dimensiones del json: ", len(to_df))

Dimensiones del json:  2


In [6]:
print("Tipo de objeto en la segunda dimensión: ", type(to_df['data']))
print("Largo de la segunda dimension: ", len(to_df["data"]))

Tipo de objeto en la segunda dimensión:  <class 'list'>
Largo de la segunda dimension:  100


Llaves de cada registro 

In [7]:
to_df["data"][0]

{'id': 47179,
 'date': '2019-01-30T00:00:00.000Z',
 'home_team': {'id': 2,
  'abbreviation': 'BOS',
  'city': 'Boston',
  'conference': 'East',
  'division': 'Atlantic',
  'full_name': 'Boston Celtics',
  'name': 'Celtics'},
 'home_team_score': 126,
 'period': 4,
 'postseason': False,
 'season': 2018,
 'status': 'Final',
 'time': ' ',
 'visitor_team': {'id': 4,
  'abbreviation': 'CHA',
  'city': 'Charlotte',
  'conference': 'East',
  'division': 'Southeast',
  'full_name': 'Charlotte Hornets',
  'name': 'Hornets'},
 'visitor_team_score': 94}

# Ejercicio 2: Organización de los datos
* En base a los datos consultados en la API, genere un objeto pd.DataFrame con la siguiente estructura:

In [8]:
def json_to_row(json_entry):
    season = json_entry['season']
    period = json_entry['period']
    home = json_entry['home_team']["full_name"]
    home_conference = json_entry['home_team']["conference"]
    home_division = json_entry['home_team']["division"]
    visitor = json_entry['visitor_team']['full_name']
    visitor_conference = json_entry['visitor_team']['conference']
    visitor_division = json_entry["visitor_team"]["division"]
    home_score = json_entry["home_team_score"]
    visitor_score = json_entry["visitor_team_score"]
    home_differential = json_entry["home_team_score"] - json_entry["visitor_team_score"]
    visitor_differential = json_entry["visitor_team_score"] - json_entry["home_team_score"]
    
    return [season,period,home,home_conference,home_division,visitor,visitor_conference,
            visitor_division,home_score,visitor_score,home_differential,visitor_differential]

In [9]:
df = pd.DataFrame(list(map(json_to_row, to_df["data"])))

In [10]:
df

,0,1,2,3,4,5,6,7,8,9,10,11
0,2018,4,Boston Celtics,East,Atlantic,Charlotte Hornets,East,Southeast,126,94,32,-32
1,2018,4,Boston Celtics,East,Atlantic,LA Clippers,West,Pacific,112,123,-11,11
2,2018,4,Philadelphia 76ers,East,Atlantic,Denver Nuggets,West,Northwest,117,110,7,-7
3,2018,4,Washington Wizards,East,Southeast,Cleveland Cavaliers,East,Central,119,106,13,-13
4,2018,4,Sacramento Kings,West,Pacific,Miami Heat,East,Southeast,102,96,6,-6
...,...,...,...,...,...,...,...,...,...,...,...,...
95,2018,4,Washington Wizards,East,Southeast,Indiana Pacers,East,Central,112,119,-7,7
96,2018,4,Brooklyn Nets,East,Atlantic,Portland Trail Blazers,West,Northwest,99,113,-14,14
97,2018,4,Cleveland Cavaliers,East,Central,Phoenix Suns,West,Pacific,111,98,13,-13
98,2018,4,Brooklyn Nets,East,Atlantic,Washington Wizards,East,Southeast,116,125,-9,9


In [11]:
df.rename(columns={0:'season',
                   1:'period',
                   2:'home',
                   3:'home_conference',
                   4:'home_division',
                   5:'visitor',
                   6:'visitor_conference',
                   7:'visitor_division',
                   8:'home_score',
                   9:'visitor_score',
                   10:'home_differential',
                   11:'visitor_differential'},
               inplace = True)

In [12]:
df

,season,period,home,home_conference,home_division,visitor,visitor_conference,visitor_division,home_score,visitor_score,home_differential,visitor_differential
0,2018,4,Boston Celtics,East,Atlantic,Charlotte Hornets,East,Southeast,126,94,32,-32
1,2018,4,Boston Celtics,East,Atlantic,LA Clippers,West,Pacific,112,123,-11,11
2,2018,4,Philadelphia 76ers,East,Atlantic,Denver Nuggets,West,Northwest,117,110,7,-7
3,2018,4,Washington Wizards,East,Southeast,Cleveland Cavaliers,East,Central,119,106,13,-13
4,2018,4,Sacramento Kings,West,Pacific,Miami Heat,East,Southeast,102,96,6,-6
...,...,...,...,...,...,...,...,...,...,...,...,...
95,2018,4,Washington Wizards,East,Southeast,Indiana Pacers,East,Central,112,119,-7,7
96,2018,4,Brooklyn Nets,East,Atlantic,Portland Trail Blazers,West,Northwest,99,113,-14,14
97,2018,4,Cleveland Cavaliers,East,Central,Phoenix Suns,West,Pacific,111,98,13,-13
98,2018,4,Brooklyn Nets,East,Atlantic,Washington Wizards,East,Southeast,116,125,-9,9


# Ejercicio 3: El efecto de jugar de local
* Genere una columna en el pd.DataFrame que identifique si el equipo que jugó de local ganó (1) o no (0).
* Repita el procedimiento para identificar si el equipo que jugó de visitante ganó (1) o no (0).
* Reporte cuáles son los primeros y últimos 5 equipos en cuanto a desempeño por jugar local.
* Reporte cuáles son los primeros y últimos 5 equipos en cuanto a desempeño por jugar de visita.

In [13]:
df["home_win"] = np.where(df["home_differential"] >= 1, 1, 0)
df["visitor_win"] = np.where(df["visitor_differential"] >= 1, 1, 0)

In [14]:
df.loc[:, 'home_differential':'visitor_win']

,home_differential,visitor_differential,home_win,visitor_win
0,32,-32,1,0
1,-11,11,0,1
2,7,-7,1,0
3,13,-13,1,0
4,6,-6,1,0
...,...,...,...,...
95,-7,7,0,1
96,-14,14,0,1
97,13,-13,1,0
98,-9,9,0,1


5 Equipos con mejor desempeño jugando local

In [15]:
home_winner_looser = df.groupby(["home"]).sum()

In [16]:
home_winner_looser["home_win"].sort_values(ascending = False).head(5)

home
Washington Wizards    4
Philadelphia 76ers    4
Detroit Pistons       4
Indiana Pacers        4
Orlando Magic         4
Name: home_win, dtype: int32

5 Equipos con peor desempeño jugando local 

In [17]:
home_winner_looser["home_win"].sort_values().head(5)

home
Phoenix Suns             0
Oklahoma City Thunder    0
Charlotte Hornets        0
New York Knicks          0
Atlanta Hawks            1
Name: home_win, dtype: int32

5 Equipos con mejor desempeño jugando de visita

In [18]:
visitor_winner_looser = df.groupby(["visitor"]).sum() 

In [19]:
home_winner_looser["visitor_win"].sort_values(ascending = False).head(5) 

home
Atlanta Hawks           4
Chicago Bulls           4
New Orleans Pelicans    4
Los Angeles Lakers      3
Memphis Grizzlies       2
Name: visitor_win, dtype: int32

5 Equipos con peor desempeño jugando de visita

In [20]:
home_winner_looser["visitor_win"].sort_values().head(5)

home
Miami Heat                0
Toronto Raptors           0
San Antonio Spurs         0
Portland Trail Blazers    0
Denver Nuggets            0
Name: visitor_win, dtype: int32

# Ejercicio 4: Obteniendo el porcentaje de ganar local y de visita
* Genere un nuevo objeto que guarde el porcentaje de juegos ganados como local por equipo.
* Repita lo mismo para los juegos donde el equipo fue visitante.
* ¿Qué equipos tienen iguales chances de ganar como visitante o local?

In [21]:
 home_winner_looser["total_matches"] = home_winner_looser["home_win"] + home_winner_looser["visitor_win"]

In [23]:
home_winner_looser["prob_home_win"] = home_winner_looser["home_win"] / home_winner_looser["total_matches"]
home_winner_looser["prob_visitor_win"] = home_winner_looser["visitor_win"] / home_winner_looser["total_matches"]

In [29]:
home_winner_looser.loc[:,"prob_home_win":"prob_visitor_win"]

,prob_home_win,prob_visitor_win
home,,
Atlanta Hawks,0.200000,0.800000
Boston Celtics,0.750000,0.250000
Brooklyn Nets,0.333333,0.666667
Charlotte Hornets,0.000000,1.000000
Chicago Bulls,0.200000,0.800000
Cleveland Cavaliers,0.500000,0.500000
Dallas Mavericks,0.666667,0.333333
Denver Nuggets,1.000000,0.000000
Detroit Pistons,1.000000,0.000000


In [32]:
same_perc = home_winner_looser[home_winner_looser[['prob_home_win','prob_visitor_win']].nunique(axis=1) == 1]

### Equipos con igual chance de ganar tanto local como de visita:

In [33]:
same_perc.loc[:,"prob_home_win":"prob_visitor_win"]

,prob_home_win,prob_visitor_win
home,,
Cleveland Cavaliers,0.5,0.5
Los Angeles Lakers,0.5,0.5
